### Сравнение тарифов нового спонсора с тарифами Partner и P4

In [3]:
import pandas as pd
from pandas import Series, DataFrame
from os import path
import roamability as rb
from os.path import join
import numpy as np

downloads = r'c:\Users\balob\Downloads\DATA'

# Country;Roaming Partner;TADIG;MOC;MTC;SMS;DATA;Service
file_tariffs_new_sponsor = 'S9_191125_old.csv'

dmi_allowed_list = 'DMI.dmi_allowed_list_export_Mon_Nov_25_2019.csv'
bss_usage = 'bss_usage_2019_08.csv'

In [4]:
# Получаем Data usage для Combined

# Like expression for Allowed List
df_al = pd.read_csv(join(downloads, dmi_allowed_list))
df_al = df_al[df_al.Sponsor.isin(['P4_Combined', 'Partner_Combined'])]
df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace={r'[Pp][Aa][rtner]?[\s\w-]*':'S1', r'[Pp]4[\s\w-]*':'S2'}, regex=True)
df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)
df_al = df_al[['SPONSOR', 'TADIG']]

# Usage in Operators from BSS
df_usage = pd.read_csv(join(downloads, bss_usage), sep=';')
cols = ['VPLMN','SPONSOR','DATA']
df_usage = df_usage.pivot_table(values='USAGE_QUANTITY', index=['REP_DATE', 'VPLMN', 'SPONSOR'], columns='USAGE_TYPE').reset_index()[cols]

df_usage.rename({'VPLMN':'TADIG', 'DATA':'DATA_VOL_MB'}, axis=1, inplace=True)
df_usage.DATA_VOL_MB = df_usage.DATA_VOL_MB/1024/1024
df_usage.columns.name=''

# Data usage in Operators from Combined
df_comb_usage = pd.merge(df_al, df_usage, how='left', on=['TADIG', 'SPONSOR'])
df_comb_usage.head(3)

,SPONSOR,TADIG,DATA_VOL_MB
0,S2,HRVT2,166090.387695
1,S1,GTMTG,NaN
2,S2,MNEMT,3158.546875


In [6]:
# Получаем данные тарифов нового Спонсора

df_new_sponsor = pd.read_csv(path.join(downloads, file_tariffs_new_sponsor), sep=',')
df_new_sponsor.drop(['Service'], axis=1, inplace=True)
df_new_sponsor.head(3)

,Country,Roaming Partner,TADIG,MOC,MTC,SMS,DATA
0,NaN,NaN,ALBAM,0.177937,0.0,0.025235,0.009874
1,NaN,NaN,ALBEM,0.198000,0.0,0.013200,0.013200
2,NaN,NaN,ALBVF,0.184882,0.0,0.019733,0.058312


In [43]:
# Очистка данных от EUR, - и преобразование нескольких TADIG в один (Опционально)

# #####################################################
#Выбрать только первый TADIG из ячейки, если их там несколько через запятую
# #####################################################

# df_new_sponsor.TADIG = df_new_sponsor.TADIG.str.extract('([A-Z]+)', expand=True)

# #####################################################
# Очистить данные от EUR с конвертацией в USD
# #####################################################

#??????????????????????????????????????????????????????
# Как можно сделать тоже самое в цикле или с помощью функции?
#??????????????????????????????????????????????????????

# df_new_sponsor.loc[df_new_sponsor.MOC.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'MOC'] =\
# df_new_sponsor.MOC.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64')*1.2

# df_new_sponsor.loc[df_new_sponsor.MTC.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'MTC'] =\
# df_new_sponsor.MTC.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64')*1.2

# df_new_sponsor.loc[df_new_sponsor.SMS.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'SMS'] =\
# df_new_sponsor.SMS.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64')*1.2

# df_new_sponsor.loc[df_new_sponsor.DATA.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'DATA'] =\
# df_new_sponsor.DATA.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64')*1.2

#??????????????????????????????????????????????????????
# Почему .str.strip() обнулил данные для DNKTD?
#??????????????????????????????????????????????????????

#df_new_sponsor.MOC = df_new_sponsor.MOC.str.strip()
#df_new_sponsor.MTC = df_new_sponsor.MTC.str.strip()
#df_new_sponsor.SMS = df_new_sponsor.SMS.str.strip()
#df_new_sponsor.DATA = df_new_sponsor.DATA.str.strip()

# df_new_sponsor[['MOC','MTC','SMS','DATA']] = df_new_sponsor[['MOC','MTC','SMS','DATA']].replace(to_replace={r'-':0}, regex=True)

df_new_sponsor[['MOC','MTC','SMS','DATA']] = df_new_sponsor[['MOC','MTC','SMS','DATA']].astype('float64')

# #####################################################
# Конвертация EUR в USD
# #####################################################

df_new_sponsor[['MOC','MTC','SMS','DATA']] = df_new_sponsor[['MOC','MTC','SMS','DATA']] * 1.2

df_new_sponsor.head(3)

,Country,Roaming Partner,TADIG,MOC,MTC,SMS,DATA
0,NaN,Monaco Maritime,AAMOM,0.62400,0.624,0.15600,2.34000
1,NaN,TMO M2M,AAZTD,0.02184,0.000,0.00708,0.00312
2,NaN,Vodafone M2M,AAZVF,0.02496,0.000,0.00240,0.00240


In [7]:
df_new_sponsor.describe()

,Country,Roaming Partner,MOC,MTC,SMS,DATA
count,0.0,0.0,527.000000,527.000000,527.000000,527.000000
mean,NaN,NaN,1.302918,0.024388,0.182176,1.952484
std,NaN,NaN,4.208973,0.144830,0.653410,9.065169
min,NaN,NaN,0.008079,0.000000,0.002640,0.002034
25%,NaN,NaN,0.090138,0.000000,0.013200,0.008436
50%,NaN,NaN,0.377445,0.000000,0.030888,0.034861
75%,NaN,NaN,0.657160,0.000369,0.179317,0.092526
max,NaN,NaN,74.609040,2.061587,12.152784,121.421334


In [8]:
# Загружаем текущие тарифы Partner и P4

sql_srt=\
'''
SELECT
t.tadig_code AS TADIG,
IIF(CURRENCY_ID=1,MO_CALL_RATE,MO_CALL_RATE*1.2) MOC,
IIF(CURRENCY_ID=1,MT_CALL_RATE,MT_CALL_RATE*1.2) MTC,
IIF(CURRENCY_ID=1,MO_SMS_RATE,MO_SMS_RATE*1.2) SMS,
IIF(CURRENCY_ID=1,DATA_RATE,DATA_RATE*1.2) DATA,
r.SPONSOR_ID
FROM ROAMING_PLAN_RULES r
LEFT JOIN RDB_TADIG_CODES t ON r.network_id=t.network_id
WHERE roaming_plan_id IN (267,268)
AND r.is_discounted = 1
AND end_date IS NULL
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_ocs = pd.read_sql_query(sql_srt, cnxn)

df_ocs_s1 = df_ocs.loc[df_ocs.SPONSOR_ID == 1, ['TADIG','MOC','MTC','SMS','DATA']]
df_ocs_s2 = df_ocs.loc[df_ocs.SPONSOR_ID == 2, ['TADIG','MOC','MTC','SMS','DATA']]
print('Partner current rates:')
display(df_ocs_s1.head(3))
print('\nP4 current rates:')
display(df_ocs_s2.head(3))

Partner current rates:


,TADIG,MOC,MTC,SMS,DATA
0,GTMSC,0.30,0.050,0.025,0.022
1,CANRW,0.05,0.015,0.008,0.007
2,CHNCU,0.15,0.060,0.200,0.015



P4 current rates:


,TADIG,MOC,MTC,SMS,DATA
7,HRVCN,0.048,0.0,0.0156,0.0072
8,INDAT,0.480,0.0,0.1680,0.0480
9,INDH1,0.480,0.0,0.1680,0.0480


In [9]:
# Объединение новых тарифов с тарифами Партнер и P4

df_compare = pd.merge(df_new_sponsor, df_ocs_s1, how='outer', on='TADIG', suffixes=['', '_S1'])
df_compare = pd.merge(df_compare, df_ocs_s2, how='outer', on='TADIG', suffixes=['', '_S2'])
df_compare.head()

,Country,Roaming Partner,TADIG,MOC,MTC,SMS,DATA,MOC_S1,MTC_S1,SMS_S1,DATA_S1,MOC_S2,MTC_S2,SMS_S2,DATA_S2
0,NaN,NaN,ALBAM,0.177937,0.0,0.025235,0.009874,NaN,NaN,NaN,NaN,0.5388,0.0,0.0624,0.0480
1,NaN,NaN,ALBEM,0.198000,0.0,0.013200,0.013200,NaN,NaN,NaN,NaN,0.4680,0.0,0.0480,0.0468
2,NaN,NaN,ALBVF,0.184882,0.0,0.019733,0.058312,0.0300,0.0,0.0062,0.0030,NaN,NaN,NaN,NaN
3,NaN,NaN,ARM01,0.158400,0.0,0.013200,0.017160,0.0250,0.0,0.0500,0.0050,0.7020,0.0,0.0180,0.0048
4,NaN,NaN,ARMOR,0.264000,0.0,0.015286,0.013530,0.1255,0.0,0.0251,0.0376,NaN,NaN,NaN,NaN


In [10]:
# Add countries

sql_str='''SELECT DISTINCT c.country_name AS COUNTRY_NAME
,SUBSTRING(t.tadig_code,1,3) AS COUNTRY_PREF
FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON c.country_id = n.country_id'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_country = pd.read_sql_query(sql_str, cnxn, coerce_float=False)
df_country.head(3)

,COUNTRY_NAME,COUNTRY_PREF
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA


In [11]:
# Добавляем Usage и название стран

df_compare_usage = df_compare.copy()
df_compare_usage['COUNTRY_PREF'] = df_compare_usage.TADIG.str.slice(0,3)
df_compare_usage = pd.merge(df_compare_usage, df_comb_usage, how='left', on='TADIG')

df_compare_usage = pd.merge(df_compare_usage, df_country, how='left', on=['COUNTRY_PREF'])
df_compare_usage.head(3)

,Country,Roaming Partner,TADIG,MOC,MTC,SMS,DATA,MOC_S1,MTC_S1,SMS_S1,DATA_S1,MOC_S2,MTC_S2,SMS_S2,DATA_S2,COUNTRY_PREF,SPONSOR,DATA_VOL_MB,COUNTRY_NAME
0,NaN,NaN,ALBAM,0.177937,0.0,0.025235,0.009874,NaN,NaN,NaN,NaN,0.5388,0.0,0.0624,0.0480,ALB,NaN,NaN,Albania
1,NaN,NaN,ALBEM,0.198000,0.0,0.013200,0.013200,NaN,NaN,NaN,NaN,0.4680,0.0,0.0480,0.0468,ALB,NaN,NaN,Albania
2,NaN,NaN,ALBVF,0.184882,0.0,0.019733,0.058312,0.03,0.0,0.0062,0.003,NaN,NaN,NaN,NaN,ALB,S1,117861.791975,Albania


In [12]:
df_compare_usage[df_compare_usage.COUNTRY_NAME.str.startswith('United Arab', na=False)]

,Country,Roaming Partner,TADIG,MOC,MTC,SMS,DATA,MOC_S1,MTC_S1,SMS_S1,DATA_S1,MOC_S2,MTC_S2,SMS_S2,DATA_S2,COUNTRY_PREF,SPONSOR,DATA_VOL_MB,COUNTRY_NAME
336,NaN,NaN,AREDU,24.715861,0.0,12.152784,120.647054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARE,NaN,NaN,United Arab Emirates
338,NaN,NaN,ARETC,1.068033,0.0,0.177766,0.271000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARE,NaN,NaN,United Arab Emirates


In [13]:
# Выгружаем отчет в файл

columns = ['COUNTRY_NAME', 'Roaming Partner','TADIG','MOC','MOC_S1','MOC_S2'
           ,'MTC','MTC_S1','MTC_S2','SMS','SMS_S1','SMS_S2','DATA','DATA_S1','DATA_S2','DATA_VOL_MB']
df_compare_usage[columns].to_csv(path.join(downloads,'result_compare.csv'), index=False)

In [14]:
# Делаем группировку для Usage

cols = ['DATA','DATA_S1','DATA_S2','DATA_VOL_MB','SPONSOR']
cols = ['DATA','DATA_S1','DATA_S2','DATA_VOL_MB','SPONSOR']
df = df_compare_usage.groupby(['COUNTRY_NAME','COUNTRY_PREF'], as_index=False)[cols].\
agg({'DATA':[('', np.min), ('_PLMN', 'nunique')],
     'DATA_S1':[('', np.min), ('_PLMN', 'nunique')],
     'DATA_S2':[('', np.min), ('_PLMN', 'nunique')],
     'DATA_VOL_MB':[('', np.sum)], 'SPONSOR':[('', 'first')]})
df.columns = [''.join(col) for col in df.columns]

def usage_calc(x):
    data = x[['DATA','DATA_S1','DATA_S2','DATA_VOL_MB','SPONSOR']]
    if x.SPONSOR == 'S1':
        x['COST_CURRENT'] = (x.DATA_VOL_MB) * x.DATA_S1
        if x.DATA < x.DATA_S1:
            x['COST_WITH_NEW'] = (x.DATA_VOL_MB) * x.DATA
        else:
            x['COST_WITH_NEW'] = x['COST_CURRENT']
    elif x.SPONSOR == 'S2':
        x['COST_CURRENT'] = (x.DATA_VOL_MB) * x.DATA_S2
        if x.DATA < x.DATA_S2:
            x['COST_WITH_NEW'] = (x.DATA_VOL_MB) * x.DATA
        else:
            x['COST_WITH_NEW'] = x['COST_CURRENT']
    return x
df_compare_usage_group = df.apply(usage_calc,axis=1)

df_compare_usage_group.head(3)

,COST_CURRENT,COST_WITH_NEW,COUNTRY_NAME,COUNTRY_PREF,DATA,DATA_PLMN,DATA_S1,DATA_S1_PLMN,DATA_S2,DATA_S2_PLMN,DATA_VOL_MB,SPONSOR
0,NaN,NaN,Afghanistan,AFG,0.007508,3,NaN,0,NaN,0,0.000000,NaN
1,353.585376,353.585376,Albania,ALB,0.009874,3,0.003,1,0.0468,2,117861.791975,S1
2,NaN,NaN,Algeria,DZA,0.026637,1,NaN,0,NaN,0,0.000000,NaN


In [15]:
cols = ['COUNTRY_NAME','DATA','DATA_PLMN','DATA_S1','DATA_S1_PLMN','DATA_S2','DATA_S2_PLMN','DATA_VOL_MB',
        'SPONSOR','COST_CURRENT','COST_WITH_NEW']
df_compare_usage_group = df_compare_usage_group[cols]
df_compare_usage_group.head(3)

,COUNTRY_NAME,DATA,DATA_PLMN,DATA_S1,DATA_S1_PLMN,DATA_S2,DATA_S2_PLMN,DATA_VOL_MB,SPONSOR,COST_CURRENT,COST_WITH_NEW
0,Afghanistan,0.007508,3,NaN,0,NaN,0,0.000000,NaN,NaN,NaN
1,Albania,0.009874,3,0.003,1,0.0468,2,117861.791975,S1,353.585376,353.585376
2,Algeria,0.026637,1,NaN,0,NaN,0,0.000000,NaN,NaN,NaN


In [16]:
df_compare_usage_group[df_compare_usage_group.COUNTRY_NAME.str.startswith('Satellite', na=False)]

,COUNTRY_NAME,DATA,DATA_PLMN,DATA_S1,DATA_S1_PLMN,DATA_S2,DATA_S2_PLMN,DATA_VOL_MB,SPONSOR,COST_CURRENT,COST_WITH_NEW
164,Satellite Networks,0.271000,2,NaN,0,NaN,0,0.000000,NaN,NaN,NaN
165,Satellite Networks,0.002112,2,0.0037,2,0.0012,2,292852.277344,S2,351.422733,351.422733


### Analysis

In [18]:
# Новые страны

df_new = df_compare_usage_group[
    (df_compare_usage_group.DATA.notnull()) &
    (df_compare_usage_group.DATA_S1.isnull()) &
    (df_compare_usage_group.DATA_S2.isnull())]
df_new.head(30)

,COUNTRY_NAME,DATA,DATA_PLMN,DATA_S1,DATA_S1_PLMN,DATA_S2,DATA_S2_PLMN,DATA_VOL_MB,SPONSOR,COST_CURRENT,COST_WITH_NEW
0,Afghanistan,0.007508,3,NaN,0,NaN,0,0.0,NaN,NaN,NaN
2,Algeria,0.026637,1,NaN,0,NaN,0,0.0,NaN,NaN,NaN
3,Andorra,12.538260,1,NaN,0,NaN,0,0.0,NaN,NaN,NaN
4,Angola,1.715061,1,NaN,0,NaN,0,0.0,NaN,NaN,NaN
12,Bahamas,0.058867,2,NaN,0,NaN,0,0.0,NaN,NaN,NaN
13,Bahrain,0.013200,2,NaN,0,NaN,0,0.0,NaN,NaN,NaN
14,Bangladesh,0.011385,2,NaN,0,NaN,0,0.0,NaN,NaN,NaN
18,Belize,0.035320,1,NaN,0,NaN,0,0.0,NaN,NaN,NaN
19,Bermuda,0.036168,1,NaN,0,NaN,0,0.0,NaN,NaN,NaN
20,Bhutan,4.697118,1,NaN,0,NaN,0,0.0,NaN,NaN,NaN


In [19]:
# Выгружаем отчет в файл

columns = ['COUNTRY_NAME', 'Roaming Partner','TADIG','MOC','MOC_S1','MOC_S2'
           ,'MTC','MTC_S1','MTC_S2','SMS','SMS_S1','SMS_S2','DATA','DATA_S1','DATA_S2','DATA_VOL_MB']

df_compare_usage[df_compare_usage.COUNTRY_NAME.isin(df_new.COUNTRY_NAME.tolist())][columns].\
to_csv(path.join(downloads,'result_new_countries.csv'), index=False)

In [20]:
# Дешевые тарифы

df_cheaper = df_compare_usage_group[
    df_compare_usage_group.COST_CURRENT > df_compare_usage_group.COST_WITH_NEW]
df_cheaper.head()

,COUNTRY_NAME,DATA,DATA_PLMN,DATA_S1,DATA_S1_PLMN,DATA_S2,DATA_S2_PLMN,DATA_VOL_MB,SPONSOR,COST_CURRENT,COST_WITH_NEW
11,Azerbaijan,0.005130,2,0.0100,1,0.01344,2,42477.863214,S1,424.778632,217.896741
22,Bosnia and Herzegov,0.054705,3,0.0627,1,0.01176,2,2656.602543,S1,166.568979,145.329073
24,Brazil,0.017271,4,0.0180,1,0.04800,1,332.854467,S1,5.991380,5.748716
39,Colombia,0.005887,2,0.0180,1,0.04800,1,27918.662188,S1,502.535919,164.348286
43,Costa Rica,0.012585,3,0.0220,1,0.04800,2,7249.250000,S1,159.483500,91.232188


In [57]:
# Выгружаем отчет в файл

columns = ['COUNTRY_NAME', 'Roaming Partner','TADIG','MOC','MOC_S1','MOC_S2'
           ,'MTC','MTC_S1','MTC_S2','SMS','SMS_S1','SMS_S2','DATA','DATA_S1','DATA_S2','DATA_VOL_MB','SPONSOR']

df_compare_usage[df_compare_usage.COUNTRY_NAME.isin(df_cheaper.COUNTRY_NAME.tolist())][columns].\
to_csv(path.join(downloads,'result_cheaper_countries.csv'), index=False)